In [23]:
import json
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import random

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, default_argument_parser, default_setup
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

from deepent.data.register_datasets import register_datasets
from deepent.config import add_deepent_config
from tools.vis import visualize_single
from tools.train_net import *

In [2]:
register_datasets(f'/home/ubuntu/RGBD-Tree-Segs-Clean/')

Registering Kelowna_train
Registering AMECT9_train
Registering SALCT1_train
Registering CPT2b_train
Registering CPT2a-n_train
Registering Kelowna_test
Registering AMECT9_test
Registering SALCT1_test
Registering CPT2b_test
Registering CPT2a-n_test


In [24]:
config_file = '../configs/deepent_fuse_rcnn_R_50_FPN.yaml'
threshold = 0.5
model = '../output/baseline_fuse_07_02_2020/model_0089999.pth'
samples = 1
dataset = 'CPT2a-n_train'
type_ = 'many'
opts = []

class Cfg_Args:
    def __init__(self, conf, tr, mod, sam, ty, opt, dts):
        self.config_file = conf
        self.threshold = tr
        self.model = mod
        self.samples = sam
        self.dataset = dts
        self.type = ty
        self.opts = []
        self.output = None
        
args = Cfg_Args(config_file, threshold, model, samples, type_, opts, dataset)

In [4]:
cfg = get_cfg()
add_deepent_config(cfg)
cfg.merge_from_file(config_file)
cfg.merge_from_list(opts)
cfg.MODEL.WEIGHTS = model 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold 
cfg.freeze()
default_setup(cfg, args)

[03/14 05:14:53 detectron2]: Rank of current process: 0. World size: 1
[03/14 05:14:54 detectron2]: Environment info:
------------------------  -------------------------------------------------------------------
sys.platform              linux
Python                    3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) [GCC 7.2.0]
Numpy                     1.15.4
Detectron2 Compiler       GCC 5.4
Detectron2 CUDA Compiler  10.1
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.3.1
PyTorch Debug Build       False
torchvision               0.4.2
CUDA available            True
GPU 0                     Tesla T4
CUDA_HOME                 /usr/local/cuda-10.1
NVCC                      Cuda compilation tools, release 10.1, V10.1.243
Pillow                    6.2.2
cv2                       4.1.2
------------------------  -------------------------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - Intel(R) Math Kernel Library Version 2018.0.3 Produ

In [22]:
from tools.predictor import RGBDPredictor
predictor = RGBDPredictor(cfg)
data = list(DatasetCatalog.get("CPT2b_test"))
metadata = MetadataCatalog.get("CPT2b_test")

RuntimeError: CUDA error: out of memory

In [19]:
visualize_single(predictor,data,metadata,"./RandomVis", 1,'./')

NameError: name 'predictor' is not defined

In [33]:
class V_args:
    def __init__(self,cfg_file, model_weights,eval_only=None):
        self.cfg_file = cfg_file
        if not eval_only == None: self.eval_only = eval_only
        self.model_weights = model_weights

In [34]:
v_args = V_args("/home/ubuntu/drone-tree-id/configs/deepent_rcnn_R_50_FPN.yaml", True, "/home/ubuntu/drone-tree-id/output/model_0034999.pth")